In [1]:
import os
from dateutil.parser import parse as dateutil_parse
from six.moves import cPickle as pickle
import time
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler as skStandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.cross_validation import train_test_split,cross_val_score

PATH = 'season_1/'
CLEAN_PATH = PATH+'clean/'

from mylib import myStandardScaler,process_order,process_traffic,get_order_group,get_traffic_group,XY_order_traffic, prediction2submit, Search_Model, DISTRICTS

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
cluster_map = pd.read_csv(CLEAN_PATH+'cluster_map.csv',index_col=0)
poi = pd.read_csv(CLEAN_PATH+'poi.csv',index_col=0)
train_order_group = pd.read_pickle(CLEAN_PATH+'train_order_group.pickle')
test_order_group = pd.read_pickle(CLEAN_PATH+'test_order_group.pickle')
train_traffic_group = pd.read_pickle(CLEAN_PATH+'train_traffic_group.pickle')
test_traffic_group = pd.read_pickle(CLEAN_PATH+'test_traffic_group.pickle')
test_target = pd.read_csv(CLEAN_PATH+'test_target.csv',index_col=0,parse_dates=True)

In [3]:
train_slot = pd.Index(sorted(train_order_group.values()[0].index.unique()))
train_slot = pd.Index(filter(lambda x: x%1000 >4,train_slot))
test_slot = test_target['datetimeslot']

In [4]:
now = time.time()

train_XY_group = dict()
for district in DISTRICTS:
    train_XY_group[district] = XY_order_traffic(district,train_order_group,train_traffic_group,train_slot)
test_XY_group = dict()
for district in DISTRICTS:
    test_XY_group[district] = XY_order_traffic(district,test_order_group,test_traffic_group,test_slot)
for district in DISTRICTS:
    scaler = myStandardScaler()
    train_XY_group[district][0] = scaler.fit_transform(train_XY_group[district][0])
    test_XY_group[district][0] = scaler.transform(test_XY_group[district][0])
    
stop = time.time()
print 'Take %02d:%02d:%02d' % ((stop-now)/3600,(stop-now)/60,(stop-now)%60)

Take 00:00:16


In [5]:
all_now = time.time()

grid_params = {'n_estimators': [444] ,'max_depth': [1,2,3], 'max_features': ['log2',None]}

# grid_params = {'n_estimators': [100] ,'max_depth': [1], 'max_features': ['log2', 'sqrt']}

search_models = {district: Search_Model(GradientBoostingRegressor) for district in DISTRICTS}
test_prediction = dict()
for district, model in search_models.items()[::-1]:
    now = time.time()
    print 'Searching %s...'%district
    model.fit(grid_params,*train_XY_group[district])
    test_prediction[district] = model.predict(test_XY_group[district][0]) - test_XY_group[district][1].fillna(0)
    with open(CLEAN_PATH+'prediction_gdbt/test_prediction_%s.pickle'%(district),'wb') as f:
        pickle.dump(test_prediction[district],f)
    stop = time.time()
    print 'Take %02d:%02d:%02d' % ((stop-now)/3600,(stop-now)/60,(stop-now)%60)
    
all_stop = time.time()
print 'Totally take %02d:%02d:%02d' % ((all_stop-all_now)/3600,(all_stop-all_now)/60,(all_stop-all_now)%60)

Searching 2350be163432e42270d2670cb3c02f80...
Best Params: {'max_features': 'log2', 'n_estimators': 444, 'max_depth': 1}
CV score: 0.228472335645
Fit (R2) score: 0.29017706677
The metrics: 0.21076353115
Take 00:00:28
Searching 4b7f6f4e2bf237b6cc58f57142bea5c0...


KeyboardInterrupt: 